# DeepSeek R1 on Databricks 試してみる
[DeepSeek R1 on Databricks](https://www.databricks.com/blog/deepseek-r1-databricks)

#### 1. カタログ・スキーマ作成


In [0]:
# カタログ情報
my_uc_catalog = 'komae_demo' # ご自分のカタログ名に変更してください
my_uc_schema = 'deep_seek'
uc_model_name = 'deepseek_r1_distilled_llama_8b'


In [0]:
# カタログ、スキーマ、ボリューム作成
spark.sql(f"CREATE CATALOG IF NOT EXISTS {my_uc_catalog};")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {my_uc_catalog}.{my_uc_schema};")

# 使うカタログ、スキーマを指定
spark.sql(f"USE CATALOG {my_uc_catalog};")
spark.sql(f"USE SCHEMA {my_uc_schema};")

print(f"カタログパス: {my_uc_catalog}")
print(f"スキーマパス: {my_uc_catalog}.{my_uc_schema}")

#### 2. モデルとトーカナイザーをロード
Llama 8Bの場合、32GB分のモデルの重みをダウンロードする必要があるため、このプロセスには数分かかります。

In [0]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Llama-8B")

#### 3. モデルとトークナイザーをtransformersモデルとして登録
mlflow.transformersを使用すると、Unity Catalogにモデルを登録するのが簡単になります。モデルサイズ（この場合は8B）とモデル名を設定するだけです。

In [0]:
import mlflow

transformers_model = {"model": model, "tokenizer": tokenizer}
task = "llm/v1/chat"

# ロードしたモデルをUnity Catalogに登録
with mlflow.start_run():
   model_info = mlflow.transformers.log_model(
       transformers_model=transformers_model,
       artifact_path="model",
       task=task,
       registered_model_name=f"{my_uc_catalog}.{my_uc_schema}.{uc_model_name}"
,
       metadata={
        "pretrained_model_name": "meta-llama/Llama-3.1-8B-Instruct",
           "databricks_model_family": "LlamaForCausalLM",
           "databricks_model_size_parameters": "8b",
       },
   )

#### 3. Model Servingにデプロイ
高度に最適化されたModel Servingエンジンを使用してこのモデルを提供するには、単純に「Serving」に移動し、登録したモデルでエンドポイントを起動  

**ServingEndPoint**
[komae_deepseek_r1_distilled_llama_8b](https://adb-984752964297111.11.azuredatabricks.net/ml/endpoints/komae_deepseek_r1_distilled_llama_8b?o=984752964297111)

<img src='https://sajpstorage.blob.core.windows.net/komae/deep_seek/model_serving.png' width='1200'/>